<a href="https://colab.research.google.com/github/DiNOV-Tokyo/yolov5/blob/main/HTML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Screenshot-to-code

In [ ]:
!pip install keras tensorflow pillow h5py 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/HTML

/content/drive/My Drive/Colab Notebooks/HTML


In [3]:
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from tensorflow.keras.utils import to_categorical
from keras.layers import Embedding, TimeDistributed, RepeatVector, LSTM, concatenate , Input, Reshape, Dense, Flatten
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
#from tensorflow.keras.preprocessing.text import Tokenizer, one_hot
#from tensorflow.keras.preprocessing.sequence import pad_sequences
#from tensorflow.keras.models import Model
#from tensorflow.keras.utils import to_categorical
#from tensorflow.keras.layers import Embedding, TimeDistributed, RepeatVector, LSTM, concatenate , Input, Reshape, Dense, Flatten
#from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
#from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
import numpy as np

In [4]:
# Load the images and preprocess them for inception-resnet
images = []
all_filenames = listdir('images/')
all_filenames.sort()
for filename in all_filenames:
    images.append(img_to_array(load_img('images/'+filename, target_size=(299, 299))))
images = np.array(images, dtype=float)
images = preprocess_input(images)

# Run the images through inception-resnet and extract the features without the classification layer
IR2 = InceptionResNetV2(weights='imagenet', include_top=False)
features = IR2.predict(images)

219070464/219055592 [==============================] - 2s 0us/step


In [5]:
# We will cap each input sequence to 100 tokens
max_caption_len = 100
# Initialize the function that will create our vocabulary 
tokenizer = Tokenizer(filters='', split=" ", lower=False)

# Read a document and return a string
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

# Load all the HTML files
X = []
all_filenames = listdir('html/')
all_filenames.sort()
for filename in all_filenames:
    X.append(load_doc('html/'+filename))

# Create the vocabulary from the html files
tokenizer.fit_on_texts(X)

# Add +1 to leave space for empty words
vocab_size = len(tokenizer.word_index) + 1
# Translate each word in text file to the matching vocabulary index
sequences = tokenizer.texts_to_sequences(X)
# The longest HTML file
max_length = max(len(s) for s in sequences)

# Intialize our final input to the model
X, y, image_data = list(), list(), list()
for img_no, seq in enumerate(sequences):
    for i in range(1, len(seq)):
        # Add the entire sequence to the input and only keep the next word for the output
        in_seq, out_seq = seq[:i], seq[i]
        # If the sentence is shorter than max_length, fill it up with empty words
        in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
        # Map the output to one-hot encoding
        out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
        # Add and image corresponding to the HTML file
        image_data.append(features[img_no])
        # Cut the input sentence to 100 tokens, and add it to the input data
        X.append(in_seq[-100:])
        y.append(out_seq)

X, y, image_data = np.array(X), np.array(y), np.array(image_data)

In [6]:
# Create the encoder
image_features = Input(shape=(8, 8, 1536,))
image_flat = Flatten()(image_features)
image_flat = Dense(128, activation='relu')(image_flat)
ir2_out = RepeatVector(max_caption_len)(image_flat)

language_input = Input(shape=(max_caption_len,))
language_model = Embedding(vocab_size, 200, input_length=max_caption_len)(language_input)
language_model = LSTM(256, return_sequences=True)(language_model)
language_model = LSTM(256, return_sequences=True)(language_model)
language_model = TimeDistributed(Dense(128, activation='relu'))(language_model)

# Create the decoder
decoder = concatenate([ir2_out, language_model])
decoder = LSTM(512, return_sequences=False)(decoder)
decoder_output = Dense(vocab_size, activation='softmax')(decoder)

# Compile the model
model = Model(inputs=[image_features, language_input], outputs=decoder_output)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [8]:
# Train the neural network
model.fit([image_data, X], y, batch_size=64, shuffle=False, epochs=70)

Epoch 1/70
37/37 [==============================] - 111s 3s/step - loss: 5.5794
Epoch 2/70
37/37 [==============================] - 111s 3s/step - loss: 5.5676
Epoch 3/70
37/37 [==============================] - 111s 3s/step - loss: 5.5537
Epoch 4/70
37/37 [==============================] - 110s 3s/step - loss: 5.5305
Epoch 5/70
37/37 [==============================] - 110s 3s/step - loss: 5.4436
Epoch 6/70
37/37 [==============================] - 111s 3s/step - loss: 5.3912
Epoch 7/70
37/37 [==============================] - 112s 3s/step - loss: 5.3484
Epoch 8/70
37/37 [==============================] - 111s 3s/step - loss: 5.4042
Epoch 9/70
37/37 [==============================] - 112s 3s/step - loss: 5.3279
Epoch 10/70
37/37 [==============================] - 110s 3s/step - loss: 5.2627
Epoch 11/70
37/37 [==============================] - 110s 3s/step - loss: 5.2134
Epoch 12/70
37/37 [==============================] - 112s 3s/step - loss: 5.1765
Epoch 13/70
37/37 [==================

In [9]:
# save the fitted model
model.save("my_model_20211130")

INFO:tensorflow:Assets written to: my_model_20211130/assets


INFO:tensorflow:Assets written to: my_model_20211130/assets


In [10]:
# map an integer to a word
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [11]:
# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
    # seed the generation process
    in_text = 'START'
    # iterate over the whole length of the sequence
    for i in range(900):
        # integer encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0][-100:]
        # pad input
        sequence = pad_sequences([sequence], maxlen=max_length)
        # predict next word
        yhat = model.predict([photo,sequence], verbose=0)
        # convert probability to integer
        yhat = np.argmax(yhat)
        # map integer to word
        word = word_for_id(yhat, tokenizer)
        # stop if we cannot map the word
        if word is None:
            break
        # append as input for generating the next word
        in_text += ' ' + word
        # Print the prediction
        print(' ' + word, end='')
        # stop if we predict the end of the sequence
        if word == 'END':
            break
    return

In [ ]:
!pwd
%cd ..


/content/drive/My Drive/Colab Notebooks/HTML/Screenshot-to-code-in-Keras
/content/drive/My Drive/Colab Notebooks/HTML


In [12]:
# Load and image, preprocess it for IR2, extract features and generate the HTML
test_image = img_to_array(load_img('images/87.jpg', target_size=(299, 299)))
test_image = np.array(test_image, dtype=float)
test_image = preprocess_input(test_image)
test_features = IR2.predict(np.array([test_image]))
#print(test_features)
ret = generate_desc(model, tokenizer, np.array(test_features), 100)

with open("result.txt", mode='w') as f:
    f.write(ret)

 Link</a></li>
 <li><a Link</a></li>
 <li><a Link</a></li>
 <li><a Link</a></li>
 <li><a Link</a></li>
 <li><a Link</a></li>
 <li><a Link</a></li>
 <li><a Link</a></li>
 <li><a Link</a></li>
 <li><a Link</a></li>
 <li><a Link</a></li>
 <li><a Link</a></li>
 <li><a Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text

TypeError: ignored

In [ ]:
print(ret)

NameError: ignored

In [ ]:
!git clone https://github.com/emilwallner/Screenshot-to-code-in-Keras.git

Cloning into 'Screenshot-to-code-in-Keras'...
remote: Enumerating objects: 4028, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 4028 (delta 2), reused 0 (delta 0), pack-reused 4022
Receiving objects: 100% (4028/4028), 50.42 MiB | 12.70 MiB/s, done.
Resolving deltas: 100% (2033/2033), done.


In [ ]:
%cd Screenshot-to-code-in-Keras


/content/drive/My Drive/Colab Notebooks/HTML/Screenshot-to-code-in-Keras


In [ ]:
!floyd login

/bin/bash: floyd: command not found
